In [1]:
import os

import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import memfd

%load_ext line_profiler

def diff_perc(image1, image2):
    gray_image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
    gray_image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
    diff = cv2.absdiff(gray_image1,gray_image2)
    return np.sum(diff)/np.prod(diff.shape)

class ABKVideoCapture(object):
    """
        Author: Ahmed Bilal Khalid
    """
    
    def __init__(self, file):
        if os.path.isfile(file):
            self.video = cv2.VideoCapture(file)
            self.frames = self.frames_generator()
            self.frames_per_second = self.video.get(cv2.CAP_PROP_FPS)
            
            width = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            self.dimensions = width, height
        else:
            raise FileNotFound

    def frames_generator(self):
        """Returns next frame from self.video"""
        
        success, image = self.video.read()  # return read_status and next_frame_image
        while success:
            success, image = self.video.read()  # return read_status and next_frame_image
            if success:
                yield image
    
    def next_frame(self):
        """Returns next frame from self.frames"""
        
        return next(self.frames)


    def create_substantial_motion_video(self, out_filename):
        THRESHOLD = 0.5

        fourcc = cv2.VideoWriter_fourcc('a', 'v', 'c', '1')
        output_video = cv2.VideoWriter(out_filename, fourcc,
                                       self.frames_per_second,
                                       self.dimensions)
        
        last_frame = self.next_frame()

        for frame in self.frames:
            diff = diff_perc(last_frame, frame)
            if diff > THRESHOLD:
                last_frame = frame
                output_video.write(frame)
        
        output_video.release()

In [2]:
video = ABKVideoCapture("Normal_Videos319_x264.mp4")

In [3]:
%lprun -f video.create_substantial_motion_video video.create_substantial_motion_video("short.mp4")